In [3]:
def moving(mylist, N):
    cumsum, moving_aves = [0], []
    for i, x in enumerate(mylist, 1):
        cumsum.append(cumsum[i-1] + x)
        #if i>=N:\n",
        incl = min(i, N)
        moving_ave = (cumsum[i] - cumsum[i-incl])/incl
        #can do stuff with moving_ave here
        moving_aves.append(moving_ave)
    return moving_aves 

In [4]:
import re
import pandas as pd
import math

df = pd.DataFrame()

frequency = 1

pattern = re.compile(r'^([0-9]*): ([0-9]+.[0-9]+), ([0-9]+.[0-9]+) avg')
testrange = { 'min' : math.inf, 'max' : 0 }
trainrange = { 'min' : math.inf }

paths = [
    '../../5_evaluate/input/results/allclasses1/allclasses.out',
]
files = [open(path) for path in paths]


data = [[] for _ in files]

try:
    nextlines = [next(lines) for lines in iter(files)]
    finished = False
except:
    finished = True
while not finished:
    
    m = [pattern.search(nextline) for nextline in nextlines]

    testerror = [float(match.group(3)) if match else 0.0 for match in m]        
    trainerror = [float(match.group(2)) if match else 0.0 for match in m]
    
    if all(m):
        for i in range(len(m)):
            data[i].append([int(m[i].group(1)), trainerror[i], testerror[i]])

    try:
        nextlines = [next(lines) for lines in iter(files)]
    except:
        finished = True

dfs = [pd.DataFrame(d) for d in data]

In [9]:
import matplotlib.pyplot as plt

plt.figure(0, figsize=(24,15))
for i in range(len(dfs)):
    df = dfs[i]
    ma = moving(df[1],126)
    
    plt.scatter(df[0], ma, s=2, label=paths[i])
plt.ylim([0,150])
plt.title("Training Error Convergence during Training of the Detecting Seven Classes Model", fontsize=30)
plt.xlabel("Iteration", fontsize=30)
plt.ylabel("Sum Squared Error", fontsize=30)
#plt.xticks(fontsize=14, rotation=90)
plt.tick_params(axis='both', which='major', labelsize=25)
#plt.xlim(30000,40000)
#plt.legend()
#plt.show()
plt.savefig('../output/errorplot.png', bbox_inches='tight')